## This notebook is used to: 
### i) Transform proper motions to Galactic coordinates
### ii) Correct for Galactic rotation 
### iii) Convert back to equatorial motions with respect to the surrounding medium

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ast
from uncertainties import ufloat, umath
from uncertainties.umath import *

import astropy.coordinates as coord
import astropy.units as u

from utils import *

#### Read the information of the source from a file

In [2]:
# reading the data from the file
filename = 'BD+43_3654.txt'
try:
    with open(filename) as f:
        data = f.read()
    data = ast.literal_eval(data)
except FileNotFoundError:
    print(f"File {filename} not found.")
    data = None

print(data)

{'ra': 308.40031523381674, 'ra_err': 0.010217067, 'dec': 43.98538212854551, 'dec_err': 0.010932052, 'parallax': 0.5823258993042517, 'parallax_err': 0.012209491, 'mu_ra_cosdec': -2.594735635507606, 'mu_ra_cosdec_err': 0.013595376, 'mu_dec': 0.7286401164024895, 'mu_dec_err': 0.014304411, 'V_r': 60, 'V_r_err': 10}


#### Define a SkyCoord object using barycentric coordinates and velocity in the ICRS

In [3]:
# Gaia DR3 data
source = coord.SkyCoord(
    ra=data['ra']*u.degree, 
    dec=data['dec']*u.degree,
    distance=(data['parallax']*u.mas).to(u.kpc, u.parallax()),
    pm_ra_cosdec=data['mu_ra_cosdec']*u.mas/u.yr,
    pm_dec=data['mu_dec']*u.mas/u.yr,
    radial_velocity=data['V_r']*u.km/u.s, 
    frame='icrs'
)

#### Get the Galactic coordinates and proper motions using astropy

In [4]:
# Save in shorter-name variables. These are all adimensional.
parallax = ufloat(data['parallax'], data['parallax_err'])
ra_err = radians(data['ra_err'])
dec_err = radians(data['dec_err'])
pm_ra_err = data['mu_ra_cosdec_err']
pm_dec_err = data['mu_dec_err']
v_r_err = data['V_r_err']

# Convert to ufloatable objects (adimensional)
ra = ufloat(source.ra.radian, ra_err)
dec = ufloat(source.dec.radian, dec_err)
pm_ra = ufloat(source.pm_ra_cosdec.to('mas/yr').value, pm_ra_err)
pm_dec = ufloat(source.pm_dec.to('mas/yr').value, pm_dec_err)
V_r = ufloat(source.galactic.radial_velocity.value, v_r_err)
print(f'Galactic coordinates: RA = {ra:.5f} rad, DEC = {dec:.5f} rad')
print(f'Proper motion: mu_ra = {pm_ra:.5f} mas/yr, mu_dec = {pm_dec:.5f} mas/yr')
print(f'Radial velocity: V_r = {V_r:.2f} km/s')

# Retrieve coordinates, distance and proper motion in Galactic coordinates. These are not adimensional. 
l, b, D = source.galactic.l, source.galactic.b, source.galactic.distance
mu_l, mu_b = source.galactic.pm_l_cosb, source.galactic.pm_b

print(f'\nDistance: {D:.3f}')
print(f'Galactic coordinates: l = {l.deg:.3f}°, b = {b.deg:.3f}°')
print(f'Proper motion: mu_l = {mu_l:.3f}, mu_b = {mu_b:.3f}')

Galactic coordinates: RA = 5.38260+/-0.00018 rad, DEC = 0.76769+/-0.00019 rad
Proper motion: mu_ra = -2.59474+/-0.01360 mas/yr, mu_dec = 0.72864+/-0.01430 mas/yr
Radial velocity: V_r = 60.00+/-10.00 km/s

Distance: 1.717 kpc
Galactic coordinates: l = 82.410°, b = 2.325°
Proper motion: mu_l = -0.958 mas / yr, mu_b = 2.519 mas / yr


#### Re-do the conversion of coordinates having error propagation
(The calculations within astropy do not work with ufloat objects. Check that the results are consistent.)

In [5]:
# Define the transformation matrix between celestial to Galactic coordinates as in: 
# https://gea.esac.esa.int/archive/documentation/GDR1/Data_processing/chap_cu3ast/sec_cu3ast_intro.html
Ag = np.array([[-0.0548755604162154,-0.8734370902348850,-0.4838350155487132],\
              [0.4941094278755837,-0.4448296299600112,0.7469822444972189],\
              [-0.8676661490190047,-0.1980763734312015,0.4559837761750669]])

dist = 1./parallax

# Intermediate vector in the Galactic frame
cra, sra = cos(ra), sin(ra)
cd, sd = cos(dec), sin(dec)
vector_gal = np.dot(Ag, np.array([cra * cd, sra * cd, sd]))

# Galactic longitude (l) and latitude (b)
l_gal = umath.atan2(vector_gal[1], vector_gal[0])
b_gal = umath.atan2(vector_gal[2], sqrt(vector_gal[0]**2 + vector_gal[1]**2))

# Calculate Galactic proper motions using the transformation matrix
mul = -sin(l_gal) * ((-Ag[0, 0] * sra + Ag[0, 1] * cra) * pm_ra - 
                      (Ag[0, 0] * cra * cd + Ag[0, 1] * sra * sd - Ag[0, 2] * cd) * pm_dec) + \
      cos(l_gal) * ((-Ag[1, 0] * sra + Ag[1, 1] * cra) * pm_ra - 
                    (Ag[1, 0] * cra * cd + Ag[1, 1] * sra * sd - Ag[1, 2] * cd) * pm_dec)

mub = -cos(l_gal) * sin(b_gal) * ((-Ag[0, 0] * sra + Ag[0, 1] * cra) * pm_ra - 
                                  (Ag[0, 0] * cra * cd + Ag[0, 1] * sra * sd - Ag[0, 2] * cd) * pm_dec) - \
      sin(l_gal) * sin(b_gal) * ((-Ag[1, 0] * sra + Ag[1, 1] * cra) * pm_ra - 
                                (Ag[1, 0] * cra * cd + Ag[1, 1] * sra * sd - Ag[1, 2] * cd) * pm_dec) + \
      cos(b_gal) * ((-Ag[2, 0] * sra + Ag[2, 1] * cra) * pm_ra - 
                    (Ag[2, 0] * cra * cd + Ag[2, 1] * sra * sd - Ag[2, 2] * cd) * pm_dec)

print('distance =', dist, 'kpc')
print(f'Galactic longitude: l = {l_gal*180/np.pi:.4f} deg')
print(f'Galactic latitude: b = {b_gal*180/np.pi:.4f} deg')
print(f'mu_l = {mul:.3f} mas/yr')
print(f'mu_b = {mub:.3f} mas/yr')

distance = 1.72+/-0.04 kpc
Galactic longitude: l = 82.4099+/-0.0098 deg
Galactic latitude: b = 2.3254+/-0.0088 deg
mu_l = -0.960+/-0.014 mas/yr
mu_b = 2.529+/-0.014 mas/yr


#### Calculate local Galactic velocity field at the position of the source

In [6]:
# Define the oort constants to use (accepted: 'C07', 'B19', 'W21' )
#constants = 'C07'
constants = 'B19'
#constants = 'W21'
A, B, C, K, U, V, W = oort_constants(constants)

# Calculate Galactic rotation velocity components
V_r0, V_l0, V_b0 = galactic_velocity(dist, b_gal, l_gal, A, B, C, K, U, V, W)

# Convert velocities to proper motions
mu_l0, mu_b0 = velocity_to_propermotion(dist, V_l0, V_b0)

print('Galactic rotation in mas/yr at the location of the source:')
print(f'(\u03BC_l)0 = {mu_l0:.3f}, (\u03BC_b)0 = {mu_b0:.3f}')

Galactic rotation in mas/yr at the location of the source:
(μ_l)0 = -5.075+/-0.018, (μ_b)0 = -0.937+/-0.019


#### Calculate the object proper motion w.r.t. the local medium

In [7]:
# Proper motions w.r.t. the surrounding medium:
mu_l_corr = mul - mu_l0
mu_b_corr = mub - mu_b0

print('\nCorrected proper motion of the source w.r.t. its environment:')
print(f'\u03BC_l = {mu_l_corr:.3f}, \u03BC_b = {mu_b_corr:.3f}')

#Calculate mu_ra, mu_dec, and V_tan. We use the transpose of the transformation matrix. 
Agt = Ag.T

# Intermediate variables for trigonometric functions
cos_l = cos(l_gal)
sin_l = sin(l_gal)
cos_b = cos(b_gal)
sin_b = sin(b_gal)

# Calculate mu_ra_corr
mu_ra_corr = -sra * ( (-Agt[0, 0] * sin_l + Agt[0, 1] * cos_l) * mu_l_corr\
    - (Agt[0, 0] * cos_l * cos_b + Agt[0, 1] * sin_l * sin_b - Agt[0, 2] * cos_b) * mu_b_corr )\
    + cra * ( (-Agt[1, 0] * sin_l + Agt[1, 1] * cos_l ) * mu_l_corr\
    - (Agt[1, 0] * cos_l * cos_b + Agt[1, 1] * sin_l * sin_b - Agt[1, 2] * cos_b) * mu_b_corr )

# Calculate mu_dec_corr
mu_dec_corr = -cra * sd * ( (-Agt[0, 0] * sin_l + Agt[0, 1] * cos_l) * mu_l_corr
    - (Agt[0, 0] * cos_l * cos_b + Agt[0, 1] * sin_l * sin_b - Agt[0, 2] * cos_b) * mu_b_corr )\
    - sra * sd * ( (-Agt[1, 0] * sin_l + Agt[1, 1] * cos_l) * mu_l_corr
    - (Agt[1, 0] * cos_l * cos_b + Agt[1, 1] * sin_l * sin_b - Agt[1, 2] * cos_b) * mu_b_corr )\
    + cd * ( (-Agt[2, 0] * sin_l + Agt[2, 1] * cos_l) * mu_l_corr
    - (Agt[2, 0] * cos_l * cos_b + Agt[2, 1] * sin_l * sin_b - Agt[2, 2] * cos_b) * mu_b_corr )

print(f'\nObserved proper motion: \u03BC_ra = {pm_ra:.1u} mas/yr, \u03BC_dec = {pm_dec:.1u} mas/yr')
print(f'Corrected proper motion: \u03BC_ra = {mu_ra_corr:.1u} mas/yr, \u03BC_dec = {mu_dec_corr:.1u} mas/yr')

# Calculate proper motion angle before and after correction for Galactic rotation 
angle = degrees( atan(pm_dec/pm_ra) )
angle_corr = degrees( atan(mu_dec_corr/mu_ra_corr) )

print(f'\nUncorrected angle [deg] = {angle:.2u}')
print(f'Corrected angle [deg] = {angle_corr:.2u}')

# Calculate the corrected tangential velocity
V_t = 4.74 * dist * sqrt(mu_l_corr**2 + mu_b_corr**2)
print(f'\nV_t [km/s] = {V_t:.2u}')

# Calculate corrected radial velocity
V_r_corr = V_r - V_r0
print(f'V_r [km/s] = {V_r_corr:.2u}')


Corrected proper motion of the source w.r.t. its environment:
μ_l = 4.116+/-0.023, μ_b = 3.466+/-0.023

Observed proper motion: μ_ra = -2.59+/-0.01 mas/yr, μ_dec = 0.73+/-0.01 mas/yr
Corrected proper motion: μ_ra = -0.08+/-0.03 mas/yr, μ_dec = 5.72+/-0.02 mas/yr

Uncorrected angle [deg] = -15.69+/-0.30
Corrected angle [deg] = -89.20+/-0.28

V_t [km/s] = 43.80+/-0.94
V_r [km/s] = 65+/-10
